Recently, I learned XGBoost by watching a series of the amazing videos hosted by [StatQuest with Josh Starmer](https://www.youtube.com/watch?v=OtD8wVaFm6E). In this notebook, I summary what I have learned.

# 1. How does XGBoost work

XGBoost builds a model by construcing a series of decision binary trees with each trees fitted to the residual of the previous trees. The output of XGBoost is the combination of these trees. The initial tree, ususally, is just a naive guess. However, after several trees are built, XGBoost can fit the traing data very well.
The residual is just the difference between the targets and predictions of the trees that have been constructed.

Then in order understand how XGBoost works, we should understand two questions:
1. how to split a leaf
1. how to make a prediction.


Since, on the surface, the other two answers are different on regression and classification,
XGBoost will be explained in Regression and Classification respectively.

Let us assume that we have made the initial prediction and built $k-1$ XGBoost trees. And now we are builting the $k$th XGBoost tree.

## 1.1 Split the leaf by using Similarity and Gain
**Denoting the residuals of the $n$ samples in a leaf in the $k$th tree by $r_i$ $(i=1,...,n)$,**
we can calculate the **Similarity Score**, $S$ of the leaf.  
For regression, it is
\begin{equation}
 S = \frac{ (\sum_{i=1}^n r_i)^2}{ \lambda + n }.
\end{equation}
For binary classification, it is
$$
 S = { {(\sum_{i=1}^n r_i)^2} \over {\lambda + \sum_{i=1}^n p^{k-1}_i(1-p^{k-1}_i)}}.
$$
Here $\lambda$ is the regularization parameter. And $p^{k-1}_i$ is the prediction of the $i$th sample in the leaf
given by the initial prediction and the $k-1$ trees that have been constructed. Or in other words, $p^{k-1}_j$ is the prediction  used to calculate the residual of the $j$th sample, $r_j$.

Then we try to find a threshold to split the leaf into two leaves (left and right)
and calculate the similarities scores on the two sub-leaves and denoting them as $S_l$ and $S_r$.
Then we can calculate the **Gain** for this split:
$$
G = S_l + S_r - S.
$$
Then we can try different thresholds to split the leaf and select the threshold with the largest gain
to construct the leaf in the tree. Repeating this procedure, we can construnct a XGBoost tree fitted to the residuals.
*I am not sure, whether $G$ must be positive in order to split a leaf*. 

## 1.2 Calculate the predictin
If this leaf is a terminal node, we can calculate the output of the leaf.
For regression, it is 
$$
  Q = {{\sum_{i=1}^n r_i} \over {\lambda + n }}.
$$
If $\lambda =0$, this is just the average of the residuals in this leaf.  
For binary classifiction, it is 
$$
  Q = { {\sum_{i=1}^n r_i} \over {\lambda + \sum_{j=1}^n p_j(1-p_j)}}.
$$

By combining the outputs of all of the XGBoost trees with the initial prediction, we can get the predition of the XGBoost model.  
For regression, it is
$$
  p = p_0 + \epsilon \times \sum_{t=1}Q_t.
$$
For binary classification, it is 
$$
  p = {1 \over {1 + e^{-Q_T}}}
$$
with
$$
  Q_T = \ln{p_0 \over {1-p_0}} + \epsilon \times \sum_{t=1}Q_t.
$$
Here, $p_0$ is the initial prediction and $\epsilon$ is the learning rate. 
And $O_t$ ($t=1,2,...$) denotes the output of the $t$-th XGBoost tree respectively.




## 1.3 Prune a tree

After a XGBoost tree is built, we prune it from bottom to root  based on the gain, $G$ and the parameter $\gamma$.
If $G < \gamma$, a split will be removed. A split would not be removed even its $G$ less than $\gamma$ 
if any of its branches is kept. 

It is noted that setting $\gamma=0$ does not mean not to prune the tree, 
since in this case, the branch with negative gain would be pruned.

## 1.4 Other parameters  
The post, [De-Mystifying XGBoost Part II](https://towardsdatascience.com/de-mystifying-xgboost-part-ii-175252dcdbc5), 
gives the explanation about the parameters in XGBoost.
- Cover:      
The cover is the denominator of the similarity score minus $\lambda$. For regression, it is
$$
 C = n.
$$
For classficiation, it is 
$$
 C = \sum_i^n p_i(1-p_i).
$$
XGBoot, defautly, sets the minimum value of $C_m$ being $1$ and does not allowed for a leaf to have
the cover less than $C_m$. Defautly, this requirement has no effect for regression. 
But for classification, the defaut setting effects. So, sometimes, an appropriate $C_m$ or the parameter, **min_child_weight**, should be selected by hand. However, usually, the default value justified the split well and should not be changed.

- $\lambda$:  
A non-zero $\lambda$ would intend to keep the number of samples in a leaf as many as possible.

- Base_score:  
The initial output. It can be set by hand when the class distribution has a big say (bias) in the final probability. 

- Max Delta Step:  
A value is used to constrain the absolute value of the score from each step (tree). Default value is zero which means no constraint.

- Scale Pos Weight:  
  For imbalanced data, the weights can be set to effect the calculation of $g$ (first derivative) and $h$ (second derivative).

# 2. Mathmatical Details
The formulas of $S$ and $O$ are different between regression and classification. 
But, underneath the surface, they have the unified form.
This can be shown by considering the construction of the $k$th XGBoot tree as minimizing the loss function $Loss$,
$$
 Loss = \sum_{i=1}^m L(y_i, p^{k-1}_i + Q_i) + {1\over2}\lambda Q_i^2.
$$
Here the subscrip $i$ denote the corresponding parameter of the $i$-th sample in the traing data,
and $\sum_{i=1}^m$ denotes the summation over the total $m$ training samples.
And $y_i$ is the target and $p^{k-1}_i$ is the prediction given by the initial prediction 
and the built $k-1$ XGBoost trees, just same as above. $Q_i$ denotes the prediction of the $k$-th 
XGBoost tree which will be built. The $k$-th XGBoost tree is just the one whose predictions minimize
$Loss$. Expanding $L(y_i, p^{k-1}_i + Q_i)$ to the second order of $Q_i$, we have
$$
  Loss \simeq \sum_{i=1}^m L(y_i, p^{k-1}_i) + \delta L
$$
with 
$$
\delta L = \sum_l [g_l Q_l + {1\over2}(h_l+\lambda) Q_l^2].
$$
and
$$
  g_l = \sum_{i_l} \frac{\partial L(y, x)} {\partial x}\bigg |_{y=y_{i_l}, x=p^{k-1}_{i_l}}
$$
$$
 h_l = \sum_{i_l} \frac{\partial^2 L(y,x)}{\partial x^2}\bigg |_{y=y_{i_l}, x=p^{k-1}_{i_l}}.
$$
Here the subscript $l$ denotes the corresponding parameter of the $l$-th terminal leaf,
and $\sum_l$ denotes the summation over all of the terminal leaves. 
The subscript $i_l$ denotes the corresponding parameter of the $i_l$-th sample in the $l$-th  terminal leaf.
And $\sum_{i_l}$ denotes the summation over all of the samples in the $l$-th leaves.

Then $Q_l$ that minimizes $Loss$ is 
$$
Q_{ml} = -\frac{g_l}{h_l + \lambda}
$$
And we have the minimum value of $\delta L $ as  
$$
  \delta L_m =  - \frac{1 \over 2}\sum_l \frac{g_l^2} {h_l + \lambda}
$$


## 2.1 Regression
For regression, we use
$$
  L(y, x) = {1\over 2} (y - x)^2.
$$
Then we have 
$$
  g_l = -\sum_{i_l}(y_{i_l} - p^{k-1}_{i_l}) = -\sum_{i_l}r_{i_l}
$$
and 
$$
  h_l = n_l + \lambda
$$
where $n_l$ is the number of the samples in the $l$-th leaf. Then we 
$$
  \delta L_m  = -{1 \over 2}\sum_l S_l
$$
with
$$
  S_l = \frac{(\sum_{i_l}r_{i_l})^2} {n_l + \lambda}
$$
This is just the Similarity Score of the $l$-th leaf.
Addtionally, we have 
$$
Q_{ml} = -\frac{g_l}{h_l + \lambda} = \frac{\sum_{i_l}r_{i_l}}{n_l + \lambda}.
$$
This is the output of the $l$-th leaf.







## 2.2 Binary classification

For binary classification, we have
$$
  L(y, x) = - [y\ln x + (1-y) \ln (1-x)] = -y \ln \frac{x}{1-x} - \ln (1-x) = -yz + \ln(1+e^z)
$$
with 
$$
  z \equiv \ln\frac{x} {1-x}.
$$
Then we can rewrite $L(y,x)$ as the function $\tilde{L}(y,z)$
$$
  \tilde{L}(y,z) = -yz + \ln(1+e^z)
$$
and
$$
  Loss = \sum_{i=1}^m \tilde{L}(y_i, z^{k-1}_i + Q_i) \simeq \sum_{i=1}^m \tilde{L}(y_i, z^{k-1}_i) + \delta L
$$
with 
$$
  \delta L = \sum_l [g_l Q_l + {1\over2}(h_l+\lambda) Q_l^2].
$$
and
$$ 
  g_l = \sum_{i_l} \frac{\partial \tilde{L}(y, z)}{\partial z}\bigg |_{y=y_{i_l}, z=z^{k-1}_{i_l}}
$$
$$
 h_l = \sum_{i_l} \frac{\partial^2 \tilde{L}(y, z)}{\partial x^2}\bigg |_{y=y_{i_l}, z=p^{k-1}_{i_l}}.
$$



Then we have 
$$
 g_l = \sum_{i_l} \left(-y_{i_l} + \frac{e^{z_{i_l}^{k-1}}}{1 + e^{z_{i_l}^{k-1}}} \right)
     = - \sum_{i_l} (y_{i_l} - p_i^{k-1}) = -\sum_{i_l} r_{i_l} 
$$
and 
$$
  h_l = \sum_{i_l} \frac{e^{z_{i_l}^{k-1}}} {\left(1 + e^{z_{i_l}^{k-1}}\right)^2} = \sum_{i_l}p^{k-1}_{i_l} (1 - p^{k-1}_{i_l})
$$

# Other Features of XGBoost
### Parallelization
XGBoost parallelizes the training process, not at tree-level but at feature-level.

### Approximate Split and Weighted Quntile Sketch with Hessian
For large dataset, sketch_eps (default at 0.03 in xgboost) to control the granularity of the buckets.
This means, defaultly, approximate 33 percentiles are used for the approximate split algorithm.
The percentiles are determined based on the weights. This is called Weighted Quntile Sketch. The weight with Hessian

### Random Forest via XGBoost
Use RF as a base learner in each iteration.

Then we have 
$$
  \delta L_m = -{1\over 2} \sum_{l} S_l
$$
with 
$$
   S_l = \frac{ (\sum_{i=1}^n r_i)^2} {\lambda + \sum_{i_l} p^{k-1}_{i_l}(1-p^{k-1}_{i_l})}.
$$
This is just the Similarity Score for binary classification.
Additionally, we have
$$
  Q_{ml} = -\frac{g_l}{h_l + \lambda} = \frac{ \sum_{i_l} r_{i_l}}{\lambda + \sum_{i_l} p^{k-1}_{i_l}(1-p^{k-1}_{i_l})}.
$$
This is the output of a leaf. Then by combining the initial ouput and the output of all of the XGBoost tress,
we have the output of the model as 
$$
Q_T = \ln \frac{p_0}{1-p0} + \epsilon \sum_t {Q_mt}
$$
However, as displayed above, this is $z^{k}$, not the prediction.
The prediction $p$ is 
$$
 Q_T = \ln \frac{p}{1-p} \Rightarrow p = \frac{1} {1 + e^{-Q_T}}.
$$
We get the result displayed in Sec. 1.2.



